## Generate Heatmaps

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/covid-ma-team/chantier-2/blob/master/HeatMaps/score_heatmap-v1.ipynb"><img src="https://colab.research.google.com/img/colab_favicon_256px.png" />Run in Google Colab</a>
</td>
</table>

In [ ]:
#generate fake data
import numpy as np
import pandas as pd
import random
import sys, os
import math
import urllib.request

import folium # last version
from folium import plugins

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import geopandas as gpd

In [ ]:
'''
#Download geojson file
data_path = os.path.join("datasets", "")
download_path = "https://raw.githubusercontent.com/covid-ma-team/chantier-2/master/HeatMaps/"
token = "?token=ACHHCCDWDBXEJ6HYUBZRAWK6YR6QQ"
os.makedirs(data_path, exist_ok=True)
for filename in ("morocco.geojson",):
    print("Downloading", filename)
    url = download_path + filename + token
    urllib.request.urlretrieve(url, data_path + filename)
    '''

In [ ]:
# Load Morocco geojson (replace with the correct one)
# We will generate points insde the Morocco Polygon    
morocco_map = gpd.read_file('./datasets/morocco.geojson')
morocco_polygon = morocco_map["geometry"][0]

In [ ]:
# generate a dictionaty data structure of user's latitude, longitude and score over time
# input the initial position, the number of users and time (integer)
def generate_random_data(init_longitude = 0.0, init_latitude = 0.0, num_users = 1, time = 1):
    data = {}
    for ts in np.arange(time):
        new_ts = {}
        for uid in np.arange(num_users):

            u = random.random()
            v = random.random()
            r = 15 # use large value to cover all morocco regions
            w = r * np.sqrt(u)
            t = 2 * np.pi * v
            epsilon_longitude = w * np.sin(t)
            epsilon_latitude = w * np.cos(t) 
            
            # costruct new point
            new_longitude = init_longitude + epsilon_longitude
            new_latitude = init_latitude + epsilon_latitude
            
            #check if the constructed point is inside Morocco polygon
            new_point = Point(new_latitude, new_longitude)
            if morocco_polygon.contains(new_point):
                new_longitude = init_longitude + epsilon_longitude
                new_latitude = init_latitude + epsilon_latitude
            else: # just keep initial position
                new_longitude = init_longitude
                new_latitude = init_latitude
            
            # create user data
            new_uid = {
                "uid":uid,
                "longitude": new_longitude, 
                "latitude": new_latitude, 
                "score": 1.0 #random.random()
                }
            new_ts[str(uid)] = new_uid
        
        # append timestamp
        data[str(ts)] = new_ts
        
    return data

In [ ]:
# initial Points  (Rabat)
INIT_LONGITUDE = 33.97
INIT_LATITUDE = -6.85

# simulation parameters
NUM_USERS = 1000
TIME = 100 # in seconds

# Generate data
data = generate_random_data( 
    init_longitude = INIT_LONGITUDE, 
    init_latitude = INIT_LATITUDE,
    num_users = NUM_USERS, 
    time = TIME)

### map_points() Helper Function 
map_points takes a single time point as a dataframe of [longitude, latitude, score] and returns a map

In [ ]:
def map_points(df, lon_col='longitude', lat_col='latitude', zoom_start=5, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lon_col: Column containing longitude (string)
        lat_col: Column containing latitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lon = df[lon_col].median()
    middle_lat = df[lat_col].median()
    

    curr_map = folium.Map(location=[middle_lon, middle_lat],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lon_col], row[lat_col]],
                                radius=pt_radius,
                                popup=row['uid'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lon_col, lat_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lon_col, lat_col, heat_map_weights_col]

        points = df[cols_to_pull].as_matrix()
        curr_map.add_children(plugins.HeatMap(points, radius=heat_map_radius))

    return curr_map

In [ ]:
# selct a time point dictionary, convert it to dataframe to be able to pass it to the map_points function
df = pd.DataFrame.from_dict(data['0'], orient='index')
df.head()

In [ ]:
map_points(df, 
           plot_points=False, 
           draw_heatmap=True, 
           heat_map_weights_normalize=True, 
           heat_map_weights_col='score',
           heat_map_radius=15)

## Heatmap Overtime

In [ ]:
# Need to reconstruct data because Folium HeatMapWithTime plugin need a list of list structure
def convert_dict_to_list_of_list(data):
    converted_data = []
    for ts in np.arange(TIME):
        list2 = []
        for key, value in data[str(ts)].items():
            list1 = []
            for k, v in value.items():
                if k!="uid":
                    list1.append(v)
            list2.append(list1)
        converted_data.append(list2)
        
    return converted_data

In [ ]:
# convert dictionary to list of list
converted_data = convert_dict_to_list_of_list(data)

# get the median position and put in the center of the map
med_lon, med_lat, med_score = np.median(np.median(np.array(converted_data), 1), 0)

In [ ]:
# create a time index
from datetime import datetime, timedelta

time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(converted_data))
]

In [ ]:
# setup the Folium map
m = folium.Map([med_lon, med_lat], tiles='OpenStreetMap', zoom_start=5)

# plot HeatMapWithTime
hm = plugins.HeatMapWithTime(
    converted_data,
    index=time_index,
    auto_play=True,
    max_opacity=0.3
)

hm.add_to(m)

m

## Trash

In [ ]:
np.random.seed(3141592)

initial_data = (
    np.random.normal(size=(NUM_USERS, 2)) * np.array([[1, 1]]) +
    np.array([[init_latitude, init_longitude]])
)

move_data = np.random.normal(size=(NUM_USERS, 2)) * 0.01

data_series = [(initial_data + move_data * i).tolist() for i in range(NUM_TS)]

score = 1  # default value
for time_entry in data_series:
    for row in time_entry:
        row.append(score)

#data_series